## Testando as informações do CSV movies

In [1]:
import pandas as pd
from datetime import datetime

movies_imdb = pd.read_csv("movies.csv", sep='|')

movies_imdb = movies_imdb.drop_duplicates(subset=['id'])

movies_imdb['genero'] = movies_imdb['genero'].replace('\\N', '')

movies_imdb['anoLancamento'] = movies_imdb['anoLancamento'].replace('\\N', pd.NA)

total_sem_genero = movies_imdb['genero'].eq('').sum()

total_sem_ano_lancamento = movies_imdb['anoLancamento'].isnull().sum()

total_filmes = len(movies_imdb)

movies_imdb['anoLancamento'] = pd.to_datetime(movies_imdb['anoLancamento'], errors='coerce')

ano_atual = datetime.now().year
filmes_filtrados = movies_imdb[
    (movies_imdb['genero'].str.contains('Drama', regex=True, na=False)) &
    (~movies_imdb['anoLancamento'].isna()) & 
    (movies_imdb['anoLancamento'].dt.year >= ano_atual - 10)
]

total_drama_romance_ultimos_10_anos = len(filmes_filtrados)

print("Total de filmes sem gênero:", total_sem_genero)
print("Total de filmes sem ano de lançamento:", total_sem_ano_lancamento)
print("Total de filmes no conjunto de dados:", total_filmes)
print("Total de filmes do gênero Drama dos últimos 10 anos:", total_drama_romance_ultimos_10_anos)


/var/folders/5q/4krf7jmj6fgfll_b9kgmff9r0000gn/T/ipykernel_1982/3791414675.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_imdb = pd.read_csv("movies.csv", sep='|')


Total de filmes sem gênero: 8511
Total de filmes sem ano de lançamento: 40
Total de filmes no conjunto de dados: 244543
Total de filmes do gênero Drama dos últimos 2 anos: 31626


In [5]:
movies_imdb.head(10)

,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
0,tt0000009,Miss Jerry,Miss Jerry,1970-01-01 00:00:00.000001894,45,Romance,5.3,200,actress,Miss Geraldine Holbrook (Miss Jerry),Blanche Bayliss,1878,1951,actress,tt0000009
3,tt0000502,Bohemios,Bohemios,1970-01-01 00:00:00.000001905,100,,4.2,14,actor,\N,Antonio del Pozo,\N,\N,actor,tt0000502
5,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1970-01-01 00:00:00.000001906,70,"Action,Adventure,Biography",6.0,797,actress,Kate Kelly,Elizabeth Tait,1879,\N,actress,tt0000574
9,tt0000591,The Prodigal Son,L'enfant prodigue,1970-01-01 00:00:00.000001907,90,Drama,5.1,20,actor,\N,Henri Gouget,\N,\N,actor,"tt3752844,tt0451296,tt0311681,tt0400955"
13,tt0000615,Robbery Under Arms,Robbery Under Arms,1970-01-01 00:00:00.000001907,\N,Drama,4.3,23,actress,\N,Rhoda Dendron,1884,1970,actress,tt0000615
19,tt0000630,Hamlet,Amleto,1970-01-01 00:00:00.000001908,\N,Drama,2.9,25,actress,Ofelia,Fernanda Negri Pouget,1889,1955,actress,"tt0003489,tt0184691,tt0167871,tt0000630"
20,tt0000679,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,1970-01-01 00:00:00.000001908,120,"Adventure,Fantasy",5.2,66,actor,The Wizard of Oz Man,L. Frank Baum,1856,1919,"writer,producer,miscellaneous","tt0004207,tt0089908,tt0032138,tt0000679"
24,tt0000862,Faldgruben,Faldgruben,1970-01-01 00:00:00.000001909,\N,,4.5,16,actor,Fader Loriot,Carl Hintz,1869,1926,actor,"tt2426100,tt2442724,tt2436368,tt0125131"
30,tt0000886,"Hamlet, Prince of Denmark",Hamlet,1970-01-01 00:00:00.000001910,\N,Drama,4.7,38,actor,Hamlet,Jean Mounet-Sully,1841,1916,actor,"tt0449855,tt0000886,tt1732180,tt0492275"
31,tt0000941,Locura de amor,Locura de amor,1970-01-01 00:00:00.000001909,45,Drama,4.5,23,actor,\N,José Durany,\N,\N,actor,"tt0011544,tt0004571,tt0191412,tt2901784"


## Testando a API do TMDB

In [12]:
import requests
import apikey
import json

API_KEY = apikey.api_key

resultados = []

for index, row in filmes_filtrados.iterrows():
    movie_id = row['id']
    
    url_external_ids = f"https://api.themoviedb.org/3/movie/{movie_id}/external_ids?api_key={API_KEY}&language=pt-BR"

    response_external_ids = requests.get(url_external_ids)
    data_external_ids = response_external_ids.json()

    if response_external_ids.status_code == 200:
        if 'imdb_id' in data_external_ids:
            imdb_id = data_external_ids['imdb_id']

            url_movie_details = f"https://api.themoviedb.org/3/movie/{imdb_id}?api_key={API_KEY}&language=pt-BR&external_source=imdb_id"

            response_movie_details = requests.get(url_movie_details)
            data_movie_details = response_movie_details.json()

            if response_movie_details.status_code == 200:
                resultados.append(data_movie_details)
            else:
                print(f"Falha na solicitação para obter detalhes do filme {movie_id}.")
        else:
            print(f"ID IMDb não encontrado nos IDs externos do filme {movie_id}.")
    else:
        print(f"Falha na solicitação para obter IDs externos do filme {movie_id}. Código de status:", response_external_ids.status_code)

with open('resultados_filmes.json', 'w') as file:
    json.dump(resultados, file, indent=4)

print("Resultados salvos com sucesso no arquivo 'resultados_filmes.json'.")


Falha na solicitação para obter IDs externos do filme tt10054946. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10188844. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10231812. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10474784. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10481912. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10545214. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10651304. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10829654. Código de status: 404


KeyboardInterrupt: 

## Testando arquivo json